In [ ]:
# ==================================
# Experiment 3
# ==================================

# estimate the reduction in the number of encountered polytopes
#when l_1 regularizaiton is utilized. Again, classifier trained on
# random data in R^2.

In [1]:
import numpy as np
from plnn import PLNN
import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable
import utilities as utils
import matplotlib.pyplot as plt

C:\Users\justin\Anaconda3\envs\DeepL\lib\site-packages\matplotlib\figure.py:98: MatplotlibDeprecationWarning: 
Adding an axes using the same arguments as a previous axes currently reuses the earlier instance.  In a future version, a new instance will always be created and returned.  Meanwhile, this warning can be suppressed, and the future behavior ensured, by passing a unique label to each axes instance.
  "Adding an axes using the same arguments as a previous axes "


In [2]:
# ==================================
# Generate Training Points
# ==================================
print('===============Generating Training Points============')
# random points at least 2r apart
m = 12
# np.random.seed(3)
x = [np.random.uniform(size=(2))]
r = 0.16
while(len(x) < m):
    p = np.random.uniform(size=(2))
    if min(np.abs(p-a).sum() for a in x) > 2*r:
        x.append(p)
# r = 0.145
epsilon = r/2

X = torch.Tensor(np.array(x))
torch.manual_seed(1)
y = (torch.rand(m)+0.5).long()

layer_sizes = [2,50,50,50,2]


===============Generating Training Points============


In [3]:
# ==================================
# Define L1-reg loss
# ==================================
def l1_loss(net):

    return sum([_.norm(p=1) for _ in net.parameters() if _.dim() > 1])

In [4]:
# ==================================
# Estimate Polytope Density as Regularization is Increased
# ==================================

# The number of unique polytopes generated is compared in the normal
# and regularized classifier cases. Results are displayed 

unique_polytopes = []
unique_polytopes_reg = []

errs = []
errs_reg = []

reg_coeffs = [1e-6, 1e-5, 1e-4, 5e-4, 1e-3, 5e-3, 1e-2, 1.0]

for reg_coeff in reg_coeffs:
    # ==================================
    # Initialize Network
    # ==================================

    print('===============Initializing Network============')
    network = PLNN(layer_sizes)
    net = network.net

    # ==================================
    # Train Network
    # ==================================

    print('===============Training Network============')
    opt = optim.Adam(net.parameters(), lr=1e-3)
    for i in range(1000):
        out = net(Variable(X))
        l = nn.CrossEntropyLoss()(out, Variable(y))
        err = (out.max(1)[1].data != y).float().mean()
        opt.zero_grad()
        (l).backward()
        opt.step()
        print('error: ', err)
        errs.append(err)

    # ==================================
    # Gather Polytopes
    # ==================================
    print('===============Gathering Polytopes============')
    num_pts = 200
    xylim = 1.0

    unique_relu_configs_list, unique_bin_acts, _, _ = utils.get_unique_relu_configs(network, xylim, num_pts)
    print('number of polytopes:', len(unique_bin_acts))
    unique_polytopes.append(len(unique_bin_acts))
    
    
    #--------------------------------------------------------------------------------------------

    # ==================================
    # Initialize Network
    # ==================================

    print('===============Initializing Network============')
    network = PLNN(layer_sizes)
    net = network.net

    # ==================================
    # Train Network
    # ==================================

    print('===============Training Network with Regularization============')
    opt = optim.Adam(net.parameters(), lr=1e-3)
    for i in range(1000):
        out = net(Variable(X))
        l = nn.CrossEntropyLoss()(out, Variable(y)).view([1])
        
#         l1_scale = 1 * 1e-3
#         l1_loss = lambda l1_scale: l1_scale * sum([_.norm(p=1) for _ in net.parameters() if _.dim() > 1])

        l1_scale = torch.Tensor([reg_coeff])
        l +=  l1_scale*l1_loss(net).view([1])
        
        err = (out.max(1)[1].data != y).float().mean()
        opt.zero_grad()
        (l).backward()
        opt.step()
        

    print('error: ', err)
    errs_reg.append(err)

    # ==================================
    # Gather Polytopes
    # ==================================
    print('===============Gathering Polytopes============')
    num_pts = 200
    xylim = 1.0

    unique_relu_configs_list_reg, unique_bin_acts, _, _ = utils.get_unique_relu_configs(network, xylim, num_pts)
    print('number of polytopes:', len(unique_bin_acts))
    unique_polytopes_reg.append(len(unique_bin_acts))
    


===============Initializing Network============
Sequential(
  (1): Linear(in_features=2, out_features=50, bias=True)
  (2): ReLU()
  (3): Linear(in_features=50, out_features=50, bias=True)
  (4): ReLU()
  (5): Linear(in_features=50, out_features=50, bias=True)
  (6): ReLU()
  (7): Linear(in_features=50, out_features=2, bias=True)
)
===============Training Network============
tensor(0.7138)
error:  tensor(0.5833)
tensor(0.7071)
error:  tensor(0.5833)
tensor(0.7009)
error:  tensor(0.5833)
tensor(0.6956)
error:  tensor(0.5833)
tensor(0.6909)
error:  tensor(0.5833)
tensor(0.6866)
error:  tensor(0.5833)
tensor(0.6824)
error:  tensor(0.4167)
tensor(0.6783)
error:  tensor(0.1667)
tensor(0.6743)
error:  tensor(0.0833)
tensor(0.6704)
error:  tensor(0.1667)
tensor(0.6665)
error:  tensor(0.2500)
tensor(0.6626)
error:  tensor(0.4167)
tensor(0.6587)
error:  tensor(0.4167)
tensor(0.6552)
error:  tensor(0.4167)
tensor(0.6516)
error:  tensor(0.4167)
tensor(0.6482)
error:  tensor(0.4167)
tensor(0.6447)

error:  tensor(0.)
tensor(0.0985)
error:  tensor(0.)
tensor(0.0966)
error:  tensor(0.)
tensor(0.0945)
error:  tensor(0.)
tensor(0.0925)
error:  tensor(0.)
tensor(0.0907)
error:  tensor(0.)
tensor(0.0888)
error:  tensor(0.)
tensor(0.0868)
error:  tensor(0.)
tensor(0.0853)
error:  tensor(0.)
tensor(0.0838)
error:  tensor(0.)
tensor(0.0819)
error:  tensor(0.)
tensor(0.0800)
error:  tensor(0.)
tensor(0.0784)
error:  tensor(0.)
tensor(0.0769)
error:  tensor(0.)
tensor(0.0752)
error:  tensor(0.)
tensor(0.0734)
error:  tensor(0.)
tensor(0.0718)
error:  tensor(0.)
tensor(0.0703)
error:  tensor(0.)
tensor(0.0689)
error:  tensor(0.)
tensor(0.0673)
error:  tensor(0.)
tensor(0.0658)
error:  tensor(0.)
tensor(0.0642)
error:  tensor(0.)
tensor(0.0630)
error:  tensor(0.)
tensor(0.0617)
error:  tensor(0.)
tensor(0.0603)
error:  tensor(0.)
tensor(0.0590)
error:  tensor(0.)
tensor(0.0575)
error:  tensor(0.)
tensor(0.0562)
error:  tensor(0.)
tensor(0.0550)
error:  tensor(0.)
tensor(0.0537)
error:  tensor

error:  tensor(0.)
tensor(0.0027)
error:  tensor(0.)
tensor(0.0027)
error:  tensor(0.)
tensor(0.0027)
error:  tensor(0.)
tensor(0.0027)
error:  tensor(0.)
tensor(0.0027)
error:  tensor(0.)
tensor(0.0026)
error:  tensor(0.)
tensor(0.0026)
error:  tensor(0.)
tensor(0.0026)
error:  tensor(0.)
tensor(0.0026)
error:  tensor(0.)
tensor(0.0026)
error:  tensor(0.)
tensor(0.0025)
error:  tensor(0.)
tensor(0.0025)
error:  tensor(0.)
tensor(0.0025)
error:  tensor(0.)
tensor(0.0025)
error:  tensor(0.)
tensor(0.0025)
error:  tensor(0.)
tensor(0.0024)
error:  tensor(0.)
tensor(0.0024)
error:  tensor(0.)
tensor(0.0024)
error:  tensor(0.)
tensor(0.0024)
error:  tensor(0.)
tensor(0.0024)
error:  tensor(0.)
tensor(0.0024)
error:  tensor(0.)
tensor(0.0023)
error:  tensor(0.)
tensor(0.0023)
error:  tensor(0.)
tensor(0.0023)
error:  tensor(0.)
tensor(0.0023)
error:  tensor(0.)
tensor(0.0023)
error:  tensor(0.)
tensor(0.0023)
error:  tensor(0.)
tensor(0.0022)
error:  tensor(0.)
tensor(0.0022)
error:  tensor

tensor(0.0008)
error:  tensor(0.)
tensor(0.0007)
error:  tensor(0.)
tensor(0.0007)
error:  tensor(0.)
tensor(0.0007)
error:  tensor(0.)
tensor(0.0007)
error:  tensor(0.)
tensor(0.0007)
error:  tensor(0.)
tensor(0.0007)
error:  tensor(0.)
tensor(0.0007)
error:  tensor(0.)
tensor(0.0007)
error:  tensor(0.)
tensor(0.0007)
error:  tensor(0.)
tensor(0.0007)
error:  tensor(0.)
tensor(0.0007)
error:  tensor(0.)
tensor(0.0007)
error:  tensor(0.)
tensor(0.0007)
error:  tensor(0.)
tensor(0.0007)
error:  tensor(0.)
tensor(0.0007)
error:  tensor(0.)
tensor(0.0007)
error:  tensor(0.)
tensor(0.0007)
error:  tensor(0.)
tensor(0.0007)
error:  tensor(0.)
tensor(0.0007)
error:  tensor(0.)
tensor(0.0007)
error:  tensor(0.)
tensor(0.0007)
error:  tensor(0.)
tensor(0.0007)
error:  tensor(0.)
tensor(0.0007)
error:  tensor(0.)
tensor(0.0007)
error:  tensor(0.)
tensor(0.0007)
error:  tensor(0.)
tensor(0.0007)
error:  tensor(0.)
tensor(0.0007)
error:  tensor(0.)
tensor(0.0007)
error:  tensor(0.)
tensor(0.0007)

error:  tensor(0.)
tensor(0.0004)
error:  tensor(0.)
tensor(0.0004)
error:  tensor(0.)
tensor(0.0003)
error:  tensor(0.)
tensor(0.0003)
error:  tensor(0.)
tensor(0.0003)
error:  tensor(0.)
tensor(0.0003)
error:  tensor(0.)
tensor(0.0003)
error:  tensor(0.)
tensor(0.0003)
error:  tensor(0.)
tensor(0.0003)
error:  tensor(0.)
tensor(0.0003)
error:  tensor(0.)
tensor(0.0003)
error:  tensor(0.)
tensor(0.0003)
error:  tensor(0.)
tensor(0.0003)
error:  tensor(0.)
tensor(0.0003)
error:  tensor(0.)
tensor(0.0003)
error:  tensor(0.)
tensor(0.0003)
error:  tensor(0.)
tensor(0.0003)
error:  tensor(0.)
tensor(0.0003)
error:  tensor(0.)
tensor(0.0003)
error:  tensor(0.)
tensor(0.0003)
error:  tensor(0.)
tensor(0.0003)
error:  tensor(0.)
tensor(0.0003)
error:  tensor(0.)
tensor(0.0003)
error:  tensor(0.)
tensor(0.0003)
error:  tensor(0.)
tensor(0.0003)
error:  tensor(0.)
tensor(0.0003)
error:  tensor(0.)
tensor(0.0003)
error:  tensor(0.)
tensor(0.0003)
error:  tensor(0.)
tensor(0.0003)
error:  tensor

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "C:\Users\justin\Anaconda3\envs\DeepL\lib\site-packages\IPython\core\interactiveshell.py", line 3267, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-4-f58054fb2082>", line 49, in <module>
    unique_relu_configs_list, unique_bin_acts, _, _ = utils.get_unique_relu_configs(network, xylim, num_pts)
  File "C:\Users\justin\Projects\geometric-certificates\utilities.py", line 373, in get_unique_relu_configs
    relu_configs = network.relu_config(pt_0, False)
  File "C:\Users\justin\Projects\geometric-certificates\plnn.py", line 75, in relu_config
    pre_relus = self.forward_by_layer(x)
  File "C:\Users\justin\Projects\geometric-certificates\plnn.py", line 179, in forward_by_layer
    pre_relus.append(x.clone())
KeyboardInterrupt

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\justin\Anaconda3\envs\DeepL\lib\site-packages\IPython\core\i

KeyboardInterrupt: 

In [ ]:
# ==================================
# Display Results
# ==================================

import matplotlib.pyplot as plt

avg = np.average(unique_polytopes)
unique_polytopes = [avg for _ in range(0, len(unique_polytopes))]
avg2 = np.average(errs)
errs = [avg2 for _ in range(0, len(errs))]

import os
cwd = os.getcwd()
plot_dir = cwd + '/plots/'
filename = plot_dir + 'l1_reg' + '.svg'

plt.figure()
ax = plt.axes()
plt.plot(reg_coeffs,unique_polytopes_reg, label='Regularized')
plt.plot(reg_coeffs,unique_polytopes, label='Unregularized')
ax.set_xscale('log')
plt.xlabel('Reg. Coefficient')
plt.ylabel('Num. Unique Polytopes')
plt.legend()
plt.savefig(filename)
plt.show()

plt.figure()
ax = plt.axes()
plt.plot(reg_coeffs,errs_reg)
plt.plot(reg_coeffs,errs)
ax.set_xscale('log')
plt.ylabel('Error_rate')
plt.xlabel('Reg. Coefficient')
plt.show()


In [ ]:
# ==================================
# Initialize Network
# ==================================

print('===============Initializing Network============')
network = PLNN(layer_sizes)
net = network.net

# ==================================
# Train Network
# ==================================

print('===============Training Network============')
opt = optim.Adam(net.parameters(), lr=1e-3)
for i in range(1000):
    out = net(Variable(X))
    l = nn.CrossEntropyLoss()(out, Variable(y))
    err = (out.max(1)[1].data != y).float().mean()
    opt.zero_grad()
    (l).backward()
    opt.step()

print('error: ', err)

# ==================================
# Gather Polytopes
# ==================================
print('===============Gathering Polytopes============')
num_pts = 200
xylim = 1.0

unique_relu_configs_list, unique_bin_acts, _, _ = utils.get_unique_relu_configs(network, xylim, num_pts)
print('number of polytopes:', len(unique_bin_acts))

# ==================================
# Visualize: classifier boundary
# ==================================

XX, YY = np.meshgrid(np.linspace(0, 1, 100), np.linspace(0, 1, 100))
X0 = Variable(torch.Tensor(np.stack([np.ravel(XX), np.ravel(YY)]).T))
y0 = network(X0)
ZZ = (y0[:,0] - y0[:,1]).resize(100, 100).data.numpy()

_, ax = plt.subplots(figsize=(8,8))
ax.contourf(XX,YY,-ZZ, cmap="coolwarm", levels=np.linspace(-1000,1000,3))
ax.scatter(X.numpy()[:,0], X.numpy()[:,1], c=y.numpy(), cmap="coolwarm", s=70)
ax.axis("equal")
ax.axis([0, 1, 0, 1])
plt.show()

# ------------------------------
# Plot Polytopes
# ------------------------------
from _polytope_ import Polytope, from_polytope_dict

polytope_list = []


for relu_configs in unique_relu_configs_list:
    polytope_dict = network.compute_polytope_config(relu_configs, True)
    polytope = from_polytope_dict(polytope_dict)
    polytope_list.append(polytope)

ax = plt.axes()
alpha = 0.6
xylim = 1.0

colors = utils.get_spaced_colors(len(polytope_list))
utils.plot_polytopes_2d(polytope_list, colors, alpha, xylim, ax)
plt.xlim(0.0, 1.0)
plt.ylim(0.0, 1.0)
plt.show()



In [ ]:
# ==================================
# Initialize Network
# ==================================

print('===============Initializing Network============')
network = PLNN(layer_sizes)
net = network.net

# ==================================
# Train Network
# ==================================

print('===============Training Network with Regularization============')
opt = optim.Adam(net.parameters(), lr=1e-3)
for i in range(1000):
    out = net(Variable(X))
    l = nn.CrossEntropyLoss()(out, Variable(y)).view([1])

    l1_scale = torch.Tensor([0.001])
    l +=  l1_scale*l1_loss(net).view([1])

    err = (out.max(1)[1].data != y).float().mean()
    opt.zero_grad()
    (l).backward()
    opt.step()


print('error: ', err)

# ==================================
# Gather Polytopes
# ==================================
print('===============Gathering Polytopes============')
num_pts = 200
xylim = 1.0

unique_relu_configs_list_reg, unique_bin_acts, _, _ = utils.get_unique_relu_configs(network, xylim, num_pts)
print('number of polytopes:', len(unique_bin_acts))

# ==================================
# Visualize: classifier boundary
# ==================================

XX, YY = np.meshgrid(np.linspace(0, 1, 100), np.linspace(0, 1, 100))
X0 = Variable(torch.Tensor(np.stack([np.ravel(XX), np.ravel(YY)]).T))
y0 = network(X0)
ZZ = (y0[:,0] - y0[:,1]).resize(100, 100).data.numpy()

_, ax = plt.subplots(figsize=(8,8))
ax.contourf(XX,YY,-ZZ, cmap="coolwarm", levels=np.linspace(-1000,1000,3))
ax.scatter(X.numpy()[:,0], X.numpy()[:,1], c=y.numpy(), cmap="coolwarm", s=70)
ax.axis("equal")
ax.axis([0, 1, 0, 1])
plt.show()

# ------------------------------
# Plot Polytopes Regularized
# ------------------------------
from _polytope_ import Polytope, from_polytope_dict

polytope_list = []


for relu_configs in unique_relu_configs_list_reg:
    polytope_dict = network.compute_polytope_config(relu_configs, True)
    polytope = from_polytope_dict(polytope_dict)
    polytope_list.append(polytope)

ax = plt.axes()
alpha = 0.6
xylim = 1.0

colors = utils.get_spaced_colors(len(polytope_list))
utils.plot_polytopes_2d(polytope_list, colors, alpha, xylim, ax)
plt.xlim(0.0, 1.0)
plt.ylim(0.0, 1.0)
plt.show()